In [1]:
import numpy as np
import pandas as pd
import math
import random
import copy

In [2]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [4]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [5]:
# define function to calculate the length of the tour 
def totalcost(tour, distance):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [6]:
# starting tour option

In [7]:
# define function to execute random tour
def random_tour(cities, distance):
    import numpy as np
    
    temp = [x for x in range(1,cities)]
    np.random.shuffle(temp)
    tour = [0] + temp + [0]
    length = totalcost(tour, distance)
    
    return (tour, length)

In [8]:
# define function to execute nearest neighbor        
def nearest_neighbor(cities, distance):
    import numpy as np

    position = 0
    tour = [0]
    length = 0
    for i in range(cities-1):
        nn = 0
        nd = np.nanmax(distance)+1
        for j in range(cities):  
            if (j not in tour) and (distance[position,j]<nd) and (i!=j):
                nd = distance[position,j]
                nn = j
        tour.append(nn)
        length = length + nd
        position = nn
    tour.append(0)
    length = length + distance[position,0]
    
    return (tour, length)

In [9]:
# define function to execute successive insertion
def succ_insert(cities, distance):
    import numpy as np

    tour = [0, 0] #start and end at node 0
    length = 0 #total length of current tour
    
    for i in range(1, cities) : #add nodes in order of 1 -> nodes-1
        length_change = np.zeros(len(tour) - 1) #list of change in length for each possible position to insert new node

        for position in range(len(tour) - 1): #iterate through possible positions in current tour
            #temp_tour = tour[: position+1] + [i] + tour[position+1 :]

            #calculate the change in length between current tour and temp tour
            pre_ins_node = tour[position]
            nex_ins_node = tour[position + 1]
            if pre_ins_node == nex_ins_node:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node]
            else:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node] - distance[pre_ins_node][nex_ins_node]

        min_position = np.argmin(length_change) #get the position which create the shortest length change
        length += length_change[min_position]
        tour = tour[: min_position+1] + [i] + tour[min_position+1 :]
        
    return (tour, length)

In [10]:
#neighborhood definition options

In [11]:
#exchange
def exchange(tour, distance, tabu_list):
    delta_list = []
    for i in range(1, len(tour)-2):
        for j in range(i+1, len(tour)-1): 
            #if ([i,j] not in tabu_list) and ([j,i] not in tabu_list):  
            delta = distance[tour[i-1],tour[i]] + distance[tour[i],tour[i+1]] + distance[tour[j-1],tour[j]] + distance[tour[j],tour[j+1]] - distance[tour[i-1],tour[j]] - distance[tour[j],tour[i+1]] - distance[tour[j-1],tour[i]] - distance[tour[i],tour[j+1]]
            delta_list.append((delta, i, j))
            #improvement = positive delta

    #exchange node i and j
    ind = np.argmax([y[0] for y in delta_list])
    ind_i = delta_list[ind][1]
    ind_j = delta_list[ind][2]
    
    new_tour = copy.deepcopy(tour)
    new_tour[ind_i] = tour[ind_j]
    new_tour[ind_j] = tour[ind_i]
    
    return(new_tour, delta_list[ind][0], [ind_i, ind_j])


In [12]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [13]:
#2opt
def two_opt(tour, distance, tabu_list):
   
    best_tour = copy.deepcopy(tour)
    best_delta = 0
    improvement = True
    ind_i = 0
    ind_j = 0
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                #if ([i,j] not in tabu_list) and ([j,i] not in tabu_list):                 
                temp_tour = tour_swap(tour, i, j)
                delta = totalcost(tour[(i-1):(j+2)], distance) - totalcost(temp_tour[(i-1):(j+2)], distance) #improvement = positive delta

                if delta > best_delta:
                    best_tour = copy.deepcopy(temp_tour)
                    best_delta = delta
                    ind_i = i
                    ind_j = j
                    improvement = True
    
    return (best_tour, best_delta, [ind_i, ind_j])

In [14]:
#main function
def tabu_search (tour, length, distance, neighbor = 'two_opt', max_length = 20, aspiration='yes',
          TimeLimit = None, non_improvement = None, max_iteration = None):

    best = length
    besttour = tour
    best_iter = 0
    print(tour)
    print(best)
    tabu = []
    
    #time limit condition
    from time import process_time 
    start_time = process_time()
     
    #max iteration condition
    iteration = 0
    non_iter = 0
    
    terminated = False
    while not terminated: #termination condition     
    
        improvement = False
        iteration += 1     
    
        next_tour = []            
        if neighbor == 'exchange':
            (next_tour, best_delta, [best_i, best_j]) = exchange(tour, distance, tabu) #end = depot
        elif neighbor == 'two_opt':
            (next_tour, best_delta, [best_i, best_j]) = two_opt(tour, distance, tabu)
        else:
            print('Invalid neighborhood definition')
            break            

        if ([best_i, best_j] not in tabu) and ([best_j, best_i] not in tabu): 
                length = length - best_delta
                tour = next_tour
                tabu.append([best_i, best_j])
        else: #move included in tabu list
            if (aspiration == 'yes') and (best_delta > 0): 
                #aspiration criterion: waive tabu status of a move if the new solution is desirable
                if [best_i, best_j] in tabu: tabu.remove([best_i, best_j])
                if [best_j, best_i] in tabu: tabu.remove([best_j, best_i])

        # tabu list must be long enough to escape from local optima
        if len(tabu) > max_length: #maintain length of tabu list less or equal to max_length
            tabu.pop(0)

        if length < best:
            best = length
            besttour=tour
            best_iter = iteration
            improvement = True
            print(best)
                
        runtime = process_time() - start_time
        
        #check termination condition
        if (TimeLimit is not None): #time elapsed termination
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True

        elif (non_improvement is not None): #max iteration without new best solution
            if improvement: 
                non_iter = 0
            else: non_iter += 1
            if non_iter >= non_improvement:
                print('Reach max number of iteration without new best solution')
                terminated = True

        elif (max_iteration is not None):
            if iteration >= max_iteration:
                print('Reach max number of iteration')
                terminated = True
        else:
            if iteration >= 25: #default value
                print('Reach default max number of iteration (20)')
                terminated = True

    print("Terminate")
    print(besttour)
    print(best)
    
    return (best, iteration, runtime)

In [15]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

for i in range(cities):
    distance[i,i] = 10000

In [16]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [17]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

In [18]:
dataset = ((cities, distance), (cities1, distance1), (cities2, distance2))
index = ['Dataset 1', 'Dataset 2', 'Dataset 3']

**Notation**  
Dataset 1 = Example with 120 cities  
Dataset 2 = Example with 152 cities  
Dataset 3 = Example with 264 cities  

### Comparision between Starting solution  
Run 3 starting solution strategies: random, nearest neighbor, successive insertion with 3 datasets, the other parameters are default (max length of tabu list = 20, neighborhood definition = 'two_opt', aspiration criteria = 'yes'). Termination criteria is set as max number of iterations = 50.

In [19]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['random', 'nearest neighbor', 'successive insertion'])

#random starting tour
for i in range(3):
    (start_tour, start_length) = random_tour(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', aspiration = 'yes', max_iteration = 25)
    value.iloc[i,0] = result

#nearest neighbor
for i in range(3):
    (start_tour, start_length) = nearest_neighbor(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', aspiration = 'yes', max_iteration = 25)
    value.iloc[i,1] = result

#successive insertion
for i in range(3):
    (start_tour, start_length) = succ_insert(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', aspiration = 'yes', max_iteration = 25)
    value.iloc[i,2] = result

[0, 101, 1, 20, 17, 68, 89, 52, 11, 33, 81, 28, 91, 51, 112, 36, 93, 54, 99, 8, 14, 4, 40, 82, 19, 111, 6, 53, 78, 88, 98, 103, 26, 18, 61, 55, 21, 71, 58, 107, 79, 110, 118, 38, 102, 77, 74, 59, 87, 72, 39, 30, 37, 108, 42, 94, 35, 105, 45, 47, 84, 116, 73, 69, 29, 80, 23, 57, 62, 97, 66, 34, 9, 106, 49, 117, 46, 2, 119, 114, 90, 104, 22, 56, 67, 3, 31, 64, 16, 65, 70, 48, 100, 85, 15, 96, 27, 75, 50, 60, 7, 109, 115, 76, 86, 63, 32, 10, 92, 95, 13, 24, 44, 113, 5, 41, 25, 83, 43, 12, 0]
55076
53352
51641
50106
48604
47221
45878
44508
43315
42132
40958
39808
38663
37489
36388
35393
34405
33348
32268
31297
30424
29572
28723
27915
27123
26208
25417
24622
23915
23185
22487
21819
21163
20609
19989
19440
18991
18480
18037
17618
17193
16819
16488
16171
15804
15475
15190
14916
14673
14439
14211
Reach max number of iteration
Terminate
[0, 110, 53, 23, 59, 15, 85, 93, 13, 73, 69, 3, 8, 50, 114, 10, 92, 52, 76, 11, 96, 87, 72, 108, 63, 20, 1, 101, 79, 26, 47, 111, 88, 54, 95, 89, 46, 35, 105, 1

88017.52424061751
86751.2775619859
85601.23436018391
82469.44079619662
81350.13213608274
80485.057354672
79888.45474357948
79332.20876785065
78775.96279212182
78219.71681639299
77813.36669790398
77410.06673062114
77205.56576559364
76892.68320409548
76729.17636469324
76677.2286891524
76633.13852025416
76338.45774562468
76025.05929409803
75878.52136504679
75837.30337570288
75642.55913659645
75605.59000398744
75589.68908792139
Reach max number of iteration
Terminate
[0, 34, 35, 33, 47, 48, 49, 75, 73, 74, 72, 50, 46, 77, 90, 95, 114, 115, 94, 91, 76, 93, 92, 123, 124, 125, 151, 149, 150, 126, 127, 122, 121, 128, 129, 148, 147, 146, 130, 131, 120, 119, 132, 133, 145, 144, 143, 134, 135, 118, 117, 136, 137, 142, 141, 140, 138, 139, 116, 110, 111, 87, 102, 101, 100, 99, 112, 113, 98, 97, 96, 89, 78, 70, 71, 52, 51, 45, 69, 68, 53, 44, 79, 88, 80, 66, 67, 55, 54, 43, 65, 64, 56, 42, 81, 86, 103, 104, 105, 106, 109, 108, 107, 84, 83, 85, 82, 62, 63, 58, 57, 41, 61, 60, 59, 40, 39, 17, 16, 8, 7

56938.766812014095
56627.94203415031
56007.567306095254
55478.341710432265
55162.113944415425
54729.658412381745
54335.400905861796
54019.173139844956
53709.271188485676
53458.13376493658
53258.13376493658
53088.41132870978
52964.8045309598
52841.197733209825
52724.04044568444
52441.197733209825
52341.197733209825
52280.321502949526
52219.766375403124
52165.29584600328
52129.66672902862
51965.29584600328
51682.45313352867
51558.18906481674
51535.90147988852
51513.6138949603
51506.23486322715
51498.85583149401
51491.476799760865
51484.40423182675
51484.404231826746
Reach max number of iteration
Terminate
[0, 40, 41, 42, 43, 44, 45, 48, 47, 46, 51, 50, 49, 55, 52, 53, 54, 126, 125, 127, 124, 123, 122, 121, 57, 56, 58, 39, 37, 36, 60, 59, 120, 119, 118, 128, 149, 153, 152, 151, 150, 230, 229, 228, 233, 231, 232, 235, 234, 227, 226, 156, 155, 154, 148, 157, 158, 159, 225, 160, 161, 147, 146, 145, 144, 164, 163, 162, 224, 223, 237, 236, 263, 239, 238, 222, 221, 167, 166, 165, 143, 142, 170,

In [ ]:
value1=value
print(value1)

#### Comment:
The performance of Nearest neighbor and Successive insertion is approximately the same. The Successive insertion works slightly worse than Nearest neighbor in dataset 1. In dataset 2, there is a mixed result, 1 test Successive insertion finds better solution and 1 test find worse one. In both tests, Successive insertion realizes tsolution quicker (fewer iterations needed) than Nearest neighbor.
However, the performance of Random starting tour is very different from the other two strategies. Random tour works worse in all datasets and in both tests, especially in large datasets. In general, the random starting tour is an unstable strategy. Therefore, it is safer to use heuristic starting tour than random one.

In [20]:
# def tabu_search (tour, length, distance, neighbor = 'random', max_length = 20, aspiration='yes',
#           TimeLimit = None, non_improvement = None, max_iteration = None):


### **Comparision between Neighborhood definition**   
Run 3 Neighborhood definitions: 2-opt and exchange with 3 datasets. The starting solution is chosen as successive insertion, based on the previous comparision. The other parameters are default (initial = 0.01, cooling factor = 0.95, number of evaluation each iteration = 10000). The termination condition is set as TimeLimit = 60 seconds per dataset per definition.